In [ ]:
print("test kernel")

ok


In [2]:
from dotenv import load_dotenv

In [3]:
from langchain_groq import ChatGroq

In [4]:
llm=ChatGroq(model="deepseek-r1-distill-llama-70b")

In [14]:
# Make sure environment variables are loaded (including GROQ_API_KEY)
load_dotenv()

# Try invoking the model again
llm.invoke("hi, what is the weather in San Francisco?")

AIMessage(content='<think>\n\n</think>\n\nHi! I don’t have real-time weather information. To find out the current weather in San Francisco, you can check a reliable weather service like Weather.com, AccuWeather, or your local news. Let me know if you need help with anything else!', additional_kwargs={}, response_metadata={'token_usage': {'completion_tokens': 57, 'prompt_tokens': 13, 'total_tokens': 70, 'completion_time': 0.231371322, 'prompt_time': 0.000444445, 'queue_time': 0.276756145, 'total_time': 0.231815767}, 'model_name': 'deepseek-r1-distill-llama-70b', 'system_fingerprint': 'fp_5d7bb96f85', 'service_tier': 'on_demand', 'finish_reason': 'stop', 'logprobs': None}, id='run--344ac14a-5756-4194-896a-6b3d67ec3912-0', usage_metadata={'input_tokens': 13, 'output_tokens': 57, 'total_tokens': 70})

In [6]:
from langchain.tools import tool

In [7]:
@tool
def multiply(a: int, b: int) -> int:
    """
    Multiply two integers.

    Args:
        a (int): The first integer.
        b (int): The second integer.

    Returns:
        int: The product of a and b.
    """
    return a * b

In [8]:
multiply

StructuredTool(name='multiply', description='Multiply two integers.\n\nArgs:\n    a (int): The first integer.\n    b (int): The second integer.\n\nReturns:\n    int: The product of a and b.', args_schema=<class 'langchain_core.utils.pydantic.multiply'>, func=<function multiply at 0x00000218F61A7B80>)

In [9]:
from langchain_core.tools import StructuredTool

In [11]:
from pydantic import BaseModel

class WeatherInput(BaseModel):
    city: str

In [12]:
def get_weather(city: str) -> str:
    """
    Get the weather for a given city.

    Args:
        city (str): The name of the city.

    Returns:
        str: A string describing the weather in the city.
    """
    return f"The weather in {city} is sunny."

In [13]:
weather_tool = StructuredTool.from_function(
    func=get_weather,
    name="get_weather",
    description="Fetches real-time weather data for a city",
    args_schema=WeatherInput,  
)

In [ ]:
class WeatherInput(BaseModel):
    city: str = Field(..., description="City name")
    units: str = Field("metric", description="metric or imperial")

class GetWeatherTool(StructuredTool):
    name: ClassVar[str] = "get_weather"           
    description: ClassVar[str] = (
        "Fetches weather data for a city"
    )
    args_schema: ClassVar[Type[BaseModel]] = WeatherInput

    def _run(self, city: str, units: str = "metric") -> str:
        return get_weather(city, units)